# Text classification

The task concentrates on content-based text classification.



1. Get acquainted with the data of the [Polish Cyberbullying detection dataset](https://huggingface.co/datasets/poleval2019_cyberbullying). 
   Pay special attention to the distribution of the positive and negative examples in the first task as well as
   distribution of the classes in the second task.


In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import *
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import csv
from imblearn.over_sampling import RandomOverSampler
import numpy as np
import random
from datasets import list_datasets, load_dataset, list_metrics, load_metric
from pycm import ConfusionMatrix
from lime import lime_text
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer
import fasttext


### Utils

In [164]:
def print_metrics(test_y,y_pred):
    print(f'f1_score={metrics.f1_score(test_y,y_pred)}')
    val = metrics.f1_score(test_y,y_pred,average='micro')
    print(f'f1_score micro={val}')
    val =metrics.f1_score(test_y,y_pred,average='macro')
    print(f'f1_score macro={val}')
    print(f'MCC={metrics.matthews_corrcoef(test_y, y_pred)}')
    print()
    print(metrics.classification_report(test_y,y_pred,target_names=['Positive','Negative']))

    print('Confusion matrix')
    print(metrics.confusion_matrix(test_y,y_pred))

In [2]:
class Statistics():
    def __init__(self):
        pass
    
    def calc_metrics(algorithm,test_y,y_pred):
        self.f1_score =  metrics.f1_score(test_y,y_pred)
        self.f1_score_micro =  metrics.f1_score(test_y,y_pred,average='micro')
        self.f1_score_macro = metrics.f1_score(test_y,y_pred,average='macro')
        self.mcc = metrics.matthews_corrcoef(test_y, y_pred)
        
        self.confusion_matrix = metrics.confusion_matrix(test_y,y_pred)
        

        
statistics = {"bayesian" : Statistics(),"fasttext" :Statistics(),"transformer" : Statistics()}



In [3]:

# print(list_datasets())

dataset = load_dataset('poleval2019_cyberbullying','task01')
print(dataset['test'][3])
print(dataset['train'][3])

train_X = dataset['train']['text']
train_y = dataset['train']['label']

test_X = dataset['test']['text']
test_y = dataset['test']['label']

oversample = RandomOverSampler(sampling_strategy='minority')

train_X, train_y = oversample.fit_resample(np.array(train_X).reshape(-1,1), np.array(train_y))
train_X = train_X.reshape(-1).tolist()
train_y=train_y.reshape(-1).tolist()


Reusing dataset poleval2019_cyber_bullying (/home/x/.cache/huggingface/datasets/poleval2019_cyber_bullying/task01/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

{'text': '@anonymized_account @anonymized_account Przecież to nawet nie jest przewrotka 😂', 'label': 0}
{'text': '@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.', 'label': 0}


In [4]:
pd.DataFrame(random.sample(list(zip(train_X,train_y)),20))

,0,1
0,@anonymized_account Powinni im naziole porządn...,1
1,@anonymized_account @anonymized_account Normal...,1
2,@anonymized_account @anonymized_account @anony...,0
3,"@anonymized_account \""A nie mówiłem?\"" 😂 No mó...",0
4,@anonymized_account @anonymized_account @anony...,0
5,@anonymized_account @anonymized_account no i ż...,0
6,@anonymized_account Takie malutkie pytanko: il...,0
7,@anonymized_account Tak to jest z pomnikami ob...,1
8,RT @anonymized_account Zamieszanie wokół Tomas...,0
9,Co ty na zjebie bez godności @anonymized_account,0


2. Train the following classifiers on the training sets (for the task 1 and the task 2)


    i. Bayesian classifier with TF * IDF weighting.
    ii. Fasttext text classifier
    iii. Transformer classifier (take into account that a number of experiments should be performed for this model).

### Bayesian classifier with TF * IDF weighting

In [166]:

tf_idf = TfidfVectorizer()


X_train_tf = tf_idf.fit_transform(train_X)
X_train_tf = tf_idf.transform(train_X)
X_test_tf = tf_idf.transform(test_X)

# print("n_samples: %d, n_features: %d" % X_train_tf.shape)
# print(X_test_tf.shape)

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf,train_y)

y_pred = naive_bayes_classifier.predict(X_test_tf)


print_metrics(test_y,y_pred)

f1_score=0.4464285714285714
f1_score micro=0.8140000000000001
f1_score macro=0.6673248626373627
MCC=0.3504588851261634

              precision    recall  f1-score   support

    Positive       0.93      0.85      0.89       866
    Negative       0.37      0.56      0.45       134

    accuracy                           0.81      1000
   macro avg       0.65      0.71      0.67      1000
weighted avg       0.85      0.81      0.83      1000

Confusion matrix
[[739 127]
 [ 59  75]]


In [161]:
def sample_tf(naive_bayes_classifier,test_X,test_y):
    testing_X = [x for (x,y)in zip(test_X,test_y) if y==1][1:10]
    testing_y = [y for (x,y)in zip(test_X,test_y) if y==1][1:10]

    test_input = tf_idf.transform(testing_X)
    result = naive_bayes_classifier.predict(test_input)
    # print(list(zip(testing_X,testing_y,result)))
    for X,y, result in zip(testing_X,testing_y,result):
        print(f'true={class_names[y]}, predicted={class_names[result]}, {X}')

sample_tf(naive_bayes_classifier,test_X,test_y)

true=harmful, predicted=non-harmful, @anonymized_account Dokładnie, pisdzielstwo nie ma prawa rozpierdalać systemu,  sądownictwa nie mając większości
true=harmful, predicted=harmful, @anonymized_account Adrian Juda, figurant WSI i  lobby żydowskiego
true=harmful, predicted=non-harmful, @anonymized_account Widać ludziom w Sączu tak pasuje. Ja bym mu w gębę napluł
true=harmful, predicted=non-harmful, @anonymized_account powinnaś odpowiedzieć za działanie na szkodę Polski i obywateli
true=harmful, predicted=non-harmful, RT @anonymized_account @anonymized_account powinnaś odpowiedzieć za działanie na szkodę Polski i obywateli
true=harmful, predicted=non-harmful, @anonymized_account @anonymized_account @anonymized_account Najbardziej to on jest wolny od mózgu.
true=harmful, predicted=harmful, @anonymized_account   Półgłówek Wieliński, wymyślił sobie półautorytaryzm!
true=harmful, predicted=harmful, RT @anonymized_account @anonymized_account   Półgłówek Wieliński, wymyślił sobie półautorytar

In [156]:
# local interpretable model-agnositc explenations LIME


def lime_explain(idx):
    class_names = ["non-harmful","harmful"]


    c = make_pipeline(tf_idf, naive_bayes_classifier)

#     print(c.predict_proba([test_X[0]]))

    explainer = LimeTextExplainer(class_names=class_names)

    exp = explainer.explain_instance(test_X[idx], c.predict_proba, num_features=6)
#     print('Document id: %d' % idx)
    print('Probability(harmful) =', c.predict_proba([test_X[idx]])[0,1])
#     print(f'True class: {class_names[test_y[idx]]} Predicted class: {class_names[y_pred[idx]]}')

    display(pd.DataFrame(exp.as_list(),columns=['word','value']))
lime_explain(0)

Probability(harmful) = 0.23346132796823493


,word,value
0,ok,-0.231657
1,Duda,0.117764
2,pięć,-0.082389
3,Spoko,-0.058828
4,im,0.057383
5,Morawieckim,-0.033708


In [157]:
def lime_explain_confusion_matrix(test_y,y_pred):
    cm = ConfusionMatrix(actual_vector=test_y, predict_vector=y_pred)
    conf_indices= cm.position()[0]

    states = ['TP','FP','FN','TN']
    for state in states:
        if len(conf_indices[state]) ==0:
            print(f'{state} no samples')
            continue
        index = conf_indices[state][0]
        print(f'{state}, True={class_names[test_y[index]]}, Pred={class_names[y_pred[index]]}, {test_X[index]}')
        lime_explain(index)
        
lime_explain_confusion_matrix(test_y,y_pred)

TP, True=non-harmful, Pred=non-harmful, @anonymized_account Spoko, jak im Duda z Morawieckim zamówią po pięć piw to wszystko będzie ok.
Probability(harmful) = 0.23346132796823493


,word,value
0,ok,-0.234264
1,Duda,0.119723
2,pięć,-0.083311
3,Spoko,-0.058784
4,im,0.058345
5,Morawieckim,-0.032837


FP, True=harmful, Pred=non-harmful, @anonymized_account Dokładnie, pisdzielstwo nie ma prawa rozpierdalać systemu,  sądownictwa nie mając większości
Probability(harmful) = 0.4202574617947113


,word,value
0,pisdzielstwo,0.193710
1,większości,-0.145243
2,systemu,0.136785
3,mając,-0.125748
4,sądownictwa,-0.099177
5,prawa,-0.024009


FN, True=non-harmful, Pred=harmful, Jajka na miękko czy na twardo? Jeśli jecie jajka ofc
Probability(harmful) = 0.7688665235325993


,word,value
0,twardo,0.171924
1,Jajka,0.120823
2,jajka,0.117454
3,Jeśli,-0.098628
4,ofc,-0.078647
5,czy,-0.030111


TN, True=harmful, Pred=harmful, @anonymized_account Tej szmaty się nie komentuje
Probability(harmful) = 0.6308802393175929


,word,value
0,szmaty,0.216023
1,komentuje,-0.096527
2,się,-0.011691
3,anonymized_account,0.007179
4,Tej,-0.005240
5,nie,-0.001681


### Fasttext classification

In [124]:

def convert_to_fasttext_format(name):
    df = pd.DataFrame(dataset[name])
    df = df[['label','text']]
    df.label = df.label.apply(lambda x: f'__label__{x}')
    return df

def tofile(df,filename):
    df.to_csv(filename, 
      index = False, 
      sep = ' ',
      header = None, 
      quoting = csv.QUOTE_NONE, 
      quotechar = "", 
      escapechar = " ")
    
df = convert_to_fasttext_format('train')
tofile(df,'train.txt')

df = convert_to_fasttext_format('test')
tofile(df,'train.txt')


In [125]:
df

,label,text
0,__label__0,"@anonymized_account Spoko, jak im Duda z Moraw..."
1,__label__0,@anonymized_account @anonymized_account Ale on...
2,__label__0,@anonymized_account No czy Prezes nie miał rac...
3,__label__0,@anonymized_account @anonymized_account Przeci...
4,__label__0,@anonymized_account @anonymized_account Owszem...
...,...,...
995,__label__0,"@anonymized_account Olej jak kto sie ubiera, p..."
996,__label__0,@anonymized_account to oczywiste byłyście dziś...
997,__label__0,@anonymized_account Duda może się przyjąć w bi...
998,__label__1,"@anonymized_account Ty jesteś jebnięty, tła ta..."


In [126]:
model = fasttext.train_supervised('train.txt',epoch=40)
y_pred = [ int(model.predict(x)[0][0][-1]) for x,y in zip(test_X,test_y)]

Read 0M words
Number of words:  4977
Number of labels: 2
Progress: 100.0% words/sec/thread:  800055 lr:  0.000000 avg.loss:  0.098123 ETA:   0h 0m 0s


In [142]:
def sample(model,test_X,test_y,number_samples_to_predict = 5):
    for label in {0,1}:
        testing_X = [x for (x,y)in zip(test_X,test_y) if y==label][1:number_samples_to_predict]
        testing_y = [y for (x,y)in zip(test_X,test_y) if y==label][1:number_samples_to_predict]
        for X,y in zip(testing_X,testing_y):
            predicted = model.predict(X)
            print(f'true={y}, predicted={predicted}, {X}')
        print()

sample(model,test_X,test_y,5)

true=0, predicted=(('__label__0',), array([0.99492949])), @anonymized_account @anonymized_account Ale on tu nie miał szans jej zagrania, a ta 'proba' to czysta prowizorka.
true=0, predicted=(('__label__0',), array([0.99901789])), @anonymized_account No czy Prezes nie miał racji, mówiąc,ze to są zdradzieckie mordy? No czy nie miał racji?😁😁
true=0, predicted=(('__label__0',), array([0.98973846])), @anonymized_account @anonymized_account Przecież to nawet nie jest przewrotka 😂
true=0, predicted=(('__label__0',), array([0.9938969])), @anonymized_account @anonymized_account Owszem podatki tak. Ale nie w takich okolicznościach. Czemu Małysza odpalili z teamu Orlen?

true=1, predicted=(('__label__1',), array([0.9689225])), @anonymized_account Dokładnie, pisdzielstwo nie ma prawa rozpierdalać systemu,  sądownictwa nie mając większości
true=1, predicted=(('__label__1',), array([0.98389184])), @anonymized_account Adrian Juda, figurant WSI i  lobby żydowskiego
true=1, predicted=(('__label__1',), 

In [150]:
print_metrics(test_y,y_pred)

f1_score=1.0
f1_score micro=1.0
f1_score macro=1.0
MCC=1.0
[[866   0]
 [  0 134]]


In [148]:
lime_explain_confusion_matrix(test_y,y_pred)

TP, True=non-harmful, Pred=non-harmful, @anonymized_account Spoko, jak im Duda z Morawieckim zamówią po pięć piw to wszystko będzie ok.
Probability(harmful) = 0.23346132796823493


,word,value
0,ok,-0.231010
1,Duda,0.120305
2,pięć,-0.084527
3,Spoko,-0.059420
4,im,0.057050
5,Morawieckim,-0.035421


FP no samples
FN no samples
TN, True=harmful, Pred=harmful, @anonymized_account Tej szmaty się nie komentuje
Probability(harmful) = 0.6308802393175929


,word,value
0,szmaty,0.217356
1,komentuje,-0.096772
2,się,-0.011056
3,anonymized_account,0.006598
4,Tej,-0.005357
5,nie,-0.002368


### Transformers

   
   
3. Compare the results of classification on the test set. Select the appropriate measures (from accuracy, F1, macro/micro F1, MCC) to compare the results.

4. Select 1 TP, 1 TN, 1 FP and 1 FN from your predictions (for the best classifier) and compare the decisions of each
   classifier on these examples using [LIME](https://github.com/marcotcr/lime).

In [30]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, rf)


NameError: name 'vectorizer' is not defined

5. Answer the following questions:


1. Which of the classifiers works the best for the task 1 and the task 2.
1. Did you achieve results comparable with the results of [PolEval Task](http://2019.poleval.pl/index.php/results/)?
1. Did you achieve results comparable with the [Klej leaderboard](https://klejbenchmark.com/leaderboard/)?
1. Describe strengths and weaknesses of each of the compared algorithms.
1. Do you think comparison of raw performance values on a single task is enough to assess the value of a given
  algorithm/model?
1. Did SHAP show that the models use valuable features/words when performing their decision?